In [1]:
DATA_LOCATION = 'data2node.txt'
VERBOSE = True
TO_FILE = True
OUTPUT_FILE = 'results.out'

In [2]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import os
import tensorflow as tf
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import keras

2023-03-29 14:38:36.793400: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 14:38:36.910834: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-29 14:38:36.910857: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-29 14:38:37.477717: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
#Debugging
def log(s):
    if VERBOSE:
        print(s)
    if TO_FILE:
        if os.path.exists(OUTPUT_FILE):
            append_write = 'a'
        else:
            append_write = 'w'
        fh = open(OUTPUT_FILE, append_write)
        fh.write(s + '\n')
        fh.close()

            


# Loading data from the files
def preprocess():
    data = []
    results = []

    with open(DATA_LOCATION) as data_file:
        for row in data_file:
            num_list = list(map(int, row.split(' ')))
            data.append(num_list[:-1])
            results.append(num_list[-1:])
    return pd.DataFrame(data), pd.DataFrame(results)


# Tiny model
def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(1000, input_dim=4,  activation='relu'),
        layers.Dense(500, activation='relu'),
        layers.Dense(1, activation='linear')
    ])

    model.compile(loss='mean_squared_error',
                  optimizer='adam')
    return model

def plot_loss(history):
    import matplotlib.pyplot as plt
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)

def plot_predictions(actual_values, predictions):
    a = plt.axes(aspect='equal')
    plt.scatter(res_tst.T, test_predictions)
    plt.xlabel('True Values [MPG]')
    plt.ylabel('Predictions [MPG]')
    lims = [0, 50]
    plt.xlim(lims)
    plt.ylim(lims)
    _ = plt.plot(lims, lims)
    
def plot_error(error):
    plt.hist(error.T, bins=25)
    plt.xlabel('Percentage error')
    plt.ylabel('Count')

In [4]:
# Load the data
data, res = preprocess()
normalizer = tf.keras.layers.Normalization(axis=-1)

eval_results = []
abs_errors = []

data_train, full_test, result_train, full_res = train_test_split(data, res, test_size=0.15)

In [5]:
# Take data in 20% increments
for i in range(15, 100, 20):
    # Get the current segment
    test_sz = i/100
    log("Current test size is: " + str(test_sz))
    _, data_slice, _, result_slice = train_test_split(data_train, result_train, test_size=test_sz)

    # Train/Test split
    log("Data size - training = " + str(len(data_slice)))

    # Build model
    normalizer.adapt(data_slice)
    dnn_model = build_and_compile_model(normalizer)

    # Train
    history = dnn_model.fit(
        data_slice,
        result_slice,
        validation_split=0.2,
        verbose=0, epochs=100)


    # Eval
    eval_res = dnn_model.evaluate(full_test, full_res, verbose=0)
    log("Eval results for test size " + str(test_sz) + " = " + str(eval_res))
    eval_results.append(eval_res)

    # Get error
    test_predictions = dnn_model.predict(full_test).flatten()
    error = (100 * test_predictions - full_res.T) / full_res.T
    abs_error = abs(error).T.sum() / len(full_test)
    log("Error for test size " + str(test_sz) + " = " + str(float(abs_error)))

    abs_errors.append(abs_error)
    
    # Plotting
    # plot_loss(history)
    # plot_predictions(test_predictions, full_res)
    # plot_error(error)

print("Absolute errors:")
print(abs_errors)
print("\n\n")

print("Evaluations:")
print(eval_results)
print("\n\n")

Current test size is: 0.15
Data size - training = 153


2023-03-29 14:38:38.524789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-29 14:38:38.525022: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-29 14:38:38.525082: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-29 14:38:38.525131: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-29 14:38:38.525181: W tensorflow/c

Eval results for test size 0.15 = nan
6/6 [==============================] - 0s 1ms/step


TypeError: cannot convert the series to <class 'float'>

In [ ]:
plot_loss(history)